# String distances



In [4]:
import nltk
import numpy as np
import pandas as pd
import editdistance

In [5]:
words = nltk.corpus.words.words()

In [6]:
pd.DataFrame(words).to_csv("words_nltk.csv")

# BK tree

## How to create a BK-tree

- take any word from your set and use it in as your root node. Add word to the tree based on their distance to the root.

- A subtree within the tree will be a node and all its downstream nodes and weights. We can represent this as a pair storing:
    - `(node: str, subtree_of_node: Dict[int, Tuple(str, ... )])`. 



### Example

- Let us consider we started a tree with the word set [**book**, **books**, **cake**] then my tree would look like this if I started by making the word **book**:

```
       1 
book ----- books
    |
    | 4
    ----cake
```

We can represent this tree in memory as a pair 
`(Node, Subtree below Node)`. That is:

```
{('book', {1: ('books', {}), 4: ('cake', {})})}
```


- Add **boo**. Now we want to add the word **boo** which is at distace 1 from book. Notice though that there is already **book** at distance 1. The BK tree has to respect that every node have all children with different distances. If there is a colission (like we have now) then the new word must become a children of the node with which there was the colition. In this case, a children of **book**. The new weight from **books** to **boo** has to be `edit_dis(books,boo)=2`.

```
       1           2
book ----- books ----- boo
    |
    | 4
    ----cake
```

#### Adding [**cape**,**cart**,**boon**,**cook**].

- Add **cape**. Notice `edit_dis(books,cape)=4` but **cake** is altready at distance 4. Therefore **cape** needs to be a child of **cake** with a weighted edge of weight `edit_dis(cake,cape)=1`.
```
       1           2
book ----- books ----- boo
    |
    | 4        1  
    ----cake ----- cape
```

- Add **cart**. Notice `edit_dis(books,cart)=4`. Therefore **cart** needs to be a children of **cake** (or any subchildren of cake). Since `edit_dis(cake,cart)=2` then :
```
       1           2
book ----- books ----- boo
    |
    | 4        1  
    ----cake ----- cape
           |
           | 2        
           ----- cart
```

- Add **boon**. Notice  `edit_dis(book,boon)=1`. Therefore **boon** has to be a descendant of **books** (because books is a son of book with weight 1). Notice `edit_dis(books,boon)=2` but there is already node **boo** at distance 2. Therefore **boon** needs to be a descendant of **boo**. Notice `edit_dis(boo,boon)=1`.

```
       1           2         1
book ----- books ----- boo ----- boon
    |                   
    | 4        1  
    ----cake ----- cape
           |
           | 2        
           ----- cart
```

- Add **cook**.  Notice  `edit_dis(book,cook)=1` but book contains a descendant at distance 1, books, so cook has to be a dascendant of books.  Notice  `edit_dis(books,cook)=2` but books has a descendant at distance 2, **boo**, so cook has to be a descendant of boo.   Notice  `edit_dis(boo,cook)=2` and boo has no other descendant at distance 2. Therefore we can add cook as descendant of boo with weight 2.

```
       1           2         1
book ----- books ----- boo ----- boon
    |                    |
    |                    |  2
    |                    ------ cook
    | 4        1  
    ----cake ----- cape
           |
           | 2        
           ----- cart
```


## How to code a BK tree

Now we would like to know how do we construct a bk-tree in python and store it. A natural way to store a tree is using many tuples concatenated. Since a tree 




##### Exercise: fill in the BKTree class to create a tree

In [21]:
edit_distance = editdistance.eval

In [22]:
words[0:10]

['A',
 'a',
 'aa',
 'aal',
 'aalii',
 'aam',
 'Aani',
 'aardvark',
 'aardwolf',
 'Aaron']

In [37]:
class BKTree:
    def __init__(self, distfn, words):
        self.distfn = distfn

        it = iter(words)
        root = next(it)
        self.tree = (root, {})

        for w in it:
            self._add_word(self.tree, w)

    def _add_word(self, parent, word):
        pword, children = parent   # ('pepe', {1:[], 2:[]})
        d = self.distfn(word, pword)
        
        # TODO BEGIN ------------------------------------------
        # call _add_word recursively to keep adding words

        # TODO END --------------------------------------------
        

In [40]:
t = BKTree(edit_distance, ['book'])
t.tree

('book', {})

In [42]:
t = BKTree(edit_distance, ['book', 'books'])
t.tree

('book', {1: ('books', {})})

In [65]:
t = BKTree(edit_distance, ['book', 'books','cake'])
t.tree

('book', {1: ('books', {}), 4: ('cake', {})})

In [70]:
words_simple = ['book', 'books', 'cake', 'boo'] 
t = BKTree(edit_distance, words_simple)
t.tree

('book', {1: ('books', {2: ('boo', {})}), 4: ('cake', {})})

In [73]:
words_simple = ['book', 'books', 'cake', 'boo', 'cape', 'cart', 'boon', 'cook'] 
t = BKTree(edit_distance, words_simple)
t.tree

('book',
 {1: ('books', {2: ('boo', {1: ('boon', {}), 2: ('cook', {})})}),
  4: ('cake', {1: ('cape', {}), 2: ('cart', {})})})

### Building a BK-Tree with many words

Let us see the cost of constructing the tree for `words` which has 23k strings

In [148]:
%%time
t = BKTree(edit_distance, [w.lower() for w in words])

CPU times: user 2.87 s, sys: 216 ms, total: 3.08 s
Wall time: 3.1 s


In [149]:
len(t.tree), len(t.tree[1]), type(t.tree[1])

(2, 24, dict)

In [150]:
t.tree[1].keys()

dict_keys([0, 1, 2, 4, 3, 7, 6, 8, 5, 10, 9, 11, 13, 15, 14, 19, 18, 16, 12, 17, 20, 21, 22, 23])

In [156]:
t.tree[1][12]

('abdominoscope',
 {1: ('abdominoscopy', {}),
  9: ('abiogenetical',
   {11: ('academization',
     {9: ('acetopiperone',
       {11: ('asperifolious',
         {9: ('epidemiology', {}),
          10: ('indefinitude', {7: ('sidelingwise', {})})}),
        10: ('autocorrosion',
         {9: ('chemiotropic', {}), 11: ('pronominalize', {})}),
        9: ('autodiffusion', {})}),
      10: ('actinomycosis',
       {10: ('ambrosiaceous',
         {11: ('apheliotropic', {}),
          6: ('arundinaceous', {}),
          7: ('chromiferous', {}),
          8: ('proditorious', {})}),
        11: ('amidothiazole',
         {11: ('bromeliaceous', {11: ('rodomontadist', {})})}),
        7: ('arteriostosis', {}),
        12: ('asperifoliate',
         {12: ('benziminazole', {}),
          10: ('cannabinaceae', {11: ('hydrosilicon', {})}),
          8: ('casuarinaceae', {}),
          9: ('irremissible', {9: ('redemonstrate', {})})}),
        9: ('cognoscitive', {})}),
      11: ('adamantoblast',
   

In [90]:
t.tree[1][23]

('epididymodeferentectomy',
 {21: ('formaldehydesulphoxylate', {}),
  23: ('pathologicopsychological', {}),
  22: ('scientificophilosophical', {}),
  20: ('tetraiodophenolphthalein', {}),
  19: ('thyroparathyroidectomize', {})})

In [91]:
branches_from_root = t.tree[1]
branch_22          = branches_from_root[22]
branch_23          = branches_from_root[23]

In [92]:
branch_22

('anthropomorphologically',
 {20: ('blepharosphincterectomy',
   {17: ('cholecystoduodenostomy', {13: ('duodenocholecystostomy', {})})}),
  19: ('choledochoduodenostomy',
   {18: ('gastroenteroanastomosis', {18: ('pseudomonocotyledonous', {})}),
    22: ('macracanthrorhynchiasis', {}),
    9: ('pancreaticoduodenostomy', {}),
    19: ('phenolsulphonephthalein', {}),
    16: ('pyopneumocholecystitis', {})}),
  21: ('electrotelethermometer',
   {21: ('formaldehydesulphoxylic', {20: ('transubstantiationalist', {})}),
    20: ('hyperconscientiousness', {20: ('pseudolamellibranchiata', {})}),
    18: ('pericardiomediastinitis', {}),
    19: ('pseudolamellibranchiate', {})}),
  18: ('hematospectrophotometer',
   {22: ('scientificogeographical', {}), 19: ('thymolsulphonephthalein', {})}),
  13: ('pathologicohistological', {}),
  14: ('philosophicotheological', {}),
  17: ('scleroticochorioiditis', {})})

In [93]:
branch_23

('epididymodeferentectomy',
 {21: ('formaldehydesulphoxylate', {}),
  23: ('pathologicopsychological', {}),
  22: ('scientificophilosophical', {}),
  20: ('tetraiodophenolphthalein', {}),
  19: ('thyroparathyroidectomize', {})})

## Searching in a BK tree

Now in order to search all words that appear at distance less or equal than a tolerance T form a query word `q` we need to visit all nodes n that are at distance D(n,q)+-N.

Let us consider word q=caqe, T=1, candidates = [], search=[book]

Select candidate **book** from search=[book]
- edit_dist(book,caqe) = 4 => candidates is not updated
    - Crawl all children of book at distance I=[4-1,4+1]=[3,5]
    - There is a single node, search=[book,cake], inside I.
    - search = [book,cake]\book = [cake]
    
Select candidate **cake** from search=[cake]
- edit_dist(cake,caqe) = 1 => candidates =[cake]
    - Crawl all children of book at distance I=[1-1,1+1]=[0,2]
    - There are 2 possible nodes, search=[cape, cart]


Select candidate **cape** from search=[cape,cart]
- edit_dist(cape,caqe) = 1 => candidates =[cake, cape]
    - Crawl all children of cape at distance I=[1-1,1+1]=[0,2]
    - Cape has no children
    - search = [cape, cart]\cape = [cart]
    
    
Select candidate **cart** from search=[cart]
- edit_dist(cart,caqe) = 2 => candidates is not updated
    - Crawl all children of cape at distance I=[1-1,1+1]=[0,2]
    - Caqe has no children
    - search = [cart]\cart = []


Select candidate... There is no candidate! stop process. 

The resulting set of possible candidates at distance 1 are: [cape,cake].

Notice that we ended up computing 4 edit distances yet we have 8 nodes.



### Making queries 

In [102]:
visited_nodes = []
class BKTree:
    def __init__(self, distfn, words):
        self.distfn = distfn

        it = iter(words)
        root = next(it)
        self.tree = (root, {})

        for i in it:
            self._add_word(self.tree, i)

    def _add_word(self, parent, word):
        pword, children = parent
        d = self.distfn(word, pword)
        # TODO BEGIN ------------------------------------------

            
        # TODO END --------------------------------------------
        
    def _search_descendants(self, parent, n, distance, query_word):
        
        node_word, children_dict = parent
        dist_to_node = distance(query_word, node_word)
        visited_nodes.append(node_word)
        results = []
        if dist_to_node <= n:
            results.append((dist_to_node, node_word))
        
        # TODO BEGIN ------------------------------------------
        # (dist - T, dist+T+1)
                    
        
        
        # TODO END --------------------------------------------

          
        return results
            
    def query(self, query_word, n):
        # sort by distance
        return sorted(self._search_descendants(self.tree, n, self.distfn, query_word))

In [103]:
%%time
t = BKTree(edit_distance,words)

CPU times: user 2.9 s, sys: 169 ms, total: 3.07 s
Wall time: 3.08 s


In [104]:
%%time
visited_nodes = []
print(t.query("senzorial", 2))

[(1, 'sensorial'), (2, 'censorial'), (2, 'mentorial'), (2, 'sectorial'), (2, 'senatorial'), (2, 'sensoria'), (2, 'tentorial')]
CPU times: user 102 ms, sys: 7.35 ms, total: 110 ms
Wall time: 109 ms


In [105]:
len(visited_nodes), len(words)

(48613, 236736)

If the edit distance is slow, the construction and search in the bktree will be slow.

Compare it with the nltk.edit_distance

In [106]:
%%time
t_nltk = BKTree(nltk.distance.edit_distance, words)

CPU times: user 2min 15s, sys: 8.48 s, total: 2min 24s
Wall time: 2min 24s


In [107]:
%%time
t_nltk.query("senzorial", 2)

CPU times: user 3.31 s, sys: 248 ms, total: 3.56 s
Wall time: 3.55 s


[(1, 'sensorial'),
 (2, 'censorial'),
 (2, 'mentorial'),
 (2, 'sectorial'),
 (2, 'senatorial'),
 (2, 'sensoria'),
 (2, 'tentorial')]

In [108]:
%%time
t.query("senzorial", 2)

CPU times: user 101 ms, sys: 7.86 ms, total: 109 ms
Wall time: 108 ms


[(1, 'sensorial'),
 (2, 'censorial'),
 (2, 'mentorial'),
 (2, 'sectorial'),
 (2, 'senatorial'),
 (2, 'sensoria'),
 (2, 'tentorial')]

### Comparing exhaustive computation vs bktree with the same edit distance function

In [114]:
from typing import List

def get_candidates_exhaustive(query:str , max_dist: int, vocabulary: List[str], sort_candidates=True):
    """
    Compute the candidate words from the vocabulary at most `max_dist` away from the input token.
    Then it filters candidates by the computed values in the distance function.
    This function is mainly used for benchmarking purposes.
    :return: list of candidate words.
    """
    query = query.lower()
    distance_token_to_words = {
        word: edit_distance(query, word) for word in vocabulary
    }
    min_dist = min(distance_token_to_words.values())
    if min_dist <= max_dist:
        if sort_candidates:
            result = sorted(
                [
                    (distance, word)
                    for word, distance in distance_token_to_words.items()
                    if distance <= max_dist
                ]
            )
        else:
            result = [
                word
                for word, distance in distance_token_to_words.items()
                if distance <= max_dist
            ]
        return result

In [194]:
word = "anthropomorphologicaly"
max_dist = 2
sort_candidates=False
        
%timeit candidates_ext = get_candidates_exhaustive(word,max_dist,words)

404 ms ± 5.26 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [195]:
candidates_ext = get_candidates_exhaustive(word,max_dist,words)
candidates_ext

[(1, 'anthropomorphological'), (1, 'anthropomorphologically')]

In [196]:
word = "anthropomorphologicaly"

%timeit candidates_ext = t.query(word, 2)

214 µs ± 1.77 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [197]:
candidates_ext = t.query(word, 2)
candidates_ext

[(1, 'anthropomorphological'), (1, 'anthropomorphologically')]

In [137]:
word = "astrologi"
max_dist = 2
sort_candidates=False
        
%timeit candidates_ext = get_candidates_exhaustive(word, max_dist, words)

319 ms ± 1.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [198]:
word = "astrologi"

%timeit t.query(word, 2)

96.2 ms ± 737 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


Notice that we have an issue with "hamer" since it is (in edit distance) as close as "hammer"  than "haler".

We would like to, somehow, choose wisely among a set of candidates.

In [111]:
candidates_bktree = t.query("hamer", 1)
candidates_bktree

[(1, 'haler'),
 (1, 'hame'),
 (1, 'hamel'),
 (1, 'hammer'),
 (1, 'hammer'),
 (1, 'hamper'),
 (1, 'harmer'),
 (1, 'hater'),
 (1, 'haver'),
 (1, 'hawer'),
 (1, 'hazer'),
 (1, 'homer'),
 (1, 'namer'),
 (1, 'shamer'),
 (1, 'tamer')]

In [113]:
%timeit  get_candidates_exhaustive(word, max_dist, words, sort_candidates)
%timeit  t.query(word, 1)

282 ms ± 2.65 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
5.86 ms ± 28.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Full comparison

In [ ]:
%timeit  get_candidates_exhaustive(word, max_dist, words, sort_candidates)
%timeit  t.query(word, 1)


### Correcting a pharse

Notice that there are several words at the same distance. Picking the correct word is not the aim for today's class. The aim is to find a small list that contains the correct word.

In [177]:
phrase = "the man wentt to the antimonarchik protest because he did not like the king"

In [182]:
%%timeit
correction = []
for w in phrase.split(" "):
    if w in words:
        correction.append(w)
    else:
        w_similar = get_candidates_exhaustive(w, 2, words, True)
        if len(w_similar)>0:
            w_corrected = w_similar[0][1]
            correction.append(w_corrected)
        else:
            # no word found, simply append the unedited word
            correction.append(w)

679 ms ± 3.16 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [192]:
print("Original:\n",phrase)
print("Corrected:\n"," ".join(correction))

Original:
 the man wentt to the antimonarchik protest because he did not like the king
Corrected:
 the man went to the antimonarchic protest because he did not like the king


In [184]:
%%timeit
correction = []
for w in phrase.split(" "):
    if w in words:
        correction.append(w)
    else:
        w_similar = t.query(w,2)
        if len(w_similar)>0:
            w_corrected = w_similar[0][1]
            correction.append(w_corrected)
        else:
            # no word found, simply append the unedited word
            correction.append(w)

117 ms ± 926 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [193]:
print("Original:\n",phrase)
print("Corrected:\n"," ".join(correction))

Original:
 the man wentt to the antimonarchik protest because he did not like the king
Corrected:
 the man went to the antimonarchic protest because he did not like the king
